In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
dataset = pd.read_csv("Market_Basket_Optimisation.csv")
dataset

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7496,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
transactions = []
for i in range(0, dataset.shape[0]):
    transactions.append([str(dataset.values[i, j]) for j in range(0, dataset.shape[1])])

In [12]:
# you only pickle once so go fg nuts
# не, реально, построение транзакций на M1 Pro занимает 6 минут. Круто.
import json
with open("transactions", "w") as fp:
    json.dump(transactions, fp)

In [13]:
import json
with open("transactions", "r") as fp:
    transactions = json.load(fp)

# И тут начинается библиотечная веселуха!

* Минимальную поддержку берем равно 0.0028 так, чтобы продукт появлялся менее чем в трех транзакция за день. Ту учитываются данные за неделю, поэтому значение поддержки 3 * 7 / 7500 = 0.0028
* Минимальное доверие ставим 0.2 просто, чтобы посмотреть, что будет
* Минимальный лифт 3 просто, чтобы посмотреть, что будет (плюс хотим, чтобы корреляция была как бы больше 1, а то чего мы тут сидим)
* Минимальная длина 2, что по логике вещей
* Максимальная длина 2, нюю, потому что лифт вычисляется для двух покупок

In [14]:
from apyori import apriori
rules = apriori(transactions=transactions, min_support=0.0028, min_cinfidence=0.2, min_lift=3, min_length=2, max_length=2)
result = list(rules)

# Придется попотеть, посмотрим на то, что покупают люди после взятия определенного первого товара :cool:

In [15]:
def daHeck(results):
    firstProd = [tuple(result[2][0][0])[0] for result in results]
    secondProd = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list(zip(firstProd, secondProd, supports, confidences, lifts))

results = pd.DataFrame(daHeck(result), columns=["First product", "Second product", "Support", "Confidence", "Lift"])

In [16]:
results.nlargest(columns = "Lift", n=20)

,First product,Second product,Support,Confidence,Lift
8,fromage blanc,honey,0.003333,0.245098,5.178128
2,chicken,light cream,0.004533,0.075556,4.843305
5,escalope,pasta,0.005867,0.073950,4.700185
15,pasta,shrimp,0.005067,0.322034,4.514494
14,olive oil,whole wheat pasta,0.008000,0.121704,4.130221
1,chicken,extra dark chocolate,0.002800,0.046667,3.888889
10,ground beef,tomato sauce,0.005333,0.054274,3.840147
7,fresh tuna,honey,0.004000,0.179641,3.795226
4,escalope,mushroom cream sauce,0.005733,0.072269,3.790327
3,cooking oil,light cream,0.002800,0.054830,3.514762


In [17]:
results.nlargest(columns = "Support", n=20)

,First product,Second product,Support,Confidence,Lift
9,ground beef,herb & pepper,0.016000,0.162822,3.291555
14,olive oil,whole wheat pasta,0.008000,0.121704,4.130221
5,escalope,pasta,0.005867,0.073950,4.700185
4,escalope,mushroom cream sauce,0.005733,0.072269,3.790327
10,ground beef,tomato sauce,0.005333,0.054274,3.840147
15,pasta,shrimp,0.005067,0.322034,4.514494
2,chicken,light cream,0.004533,0.075556,4.843305
6,fresh bread,tomato juice,0.004267,0.099071,3.273278
7,fresh tuna,honey,0.004000,0.179641,3.795226
0,brownies,cottage cheese,0.003467,0.102767,3.238450


# ECLAT

In [18]:
from pyECLAT import ECLAT

eclat = ECLAT(pd.DataFrame(transactions))
eclat.df_bin

,burger sauce,sandwich,gums,bacon,cake,cereals,pancakes,yams,escalope,water spray,...,butter,meatballs,barbecue sauce,protein bar,green tea,cooking oil,green grapes,babies food,salt,vegetables mix
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [19]:
ruleIndices, ruleSupports = eclat.fit(min_support=0.0028, min_combination=2, max_combination=2, separator= ' & ', verbose=True)

Combination 2 by 2


6555it [00:12, 535.19it/s]


In [20]:
result = pd.DataFrame(ruleSupports.items(), columns=['Item', 'Support'])
result.sort_values(by=['Support'], ascending=False)

,Item,Support
657,spaghetti & mineral water,0.059733
301,chocolate & mineral water,0.052667
621,eggs & mineral water,0.050933
434,milk & mineral water,0.048000
373,ground beef & mineral water,0.040933
...,...,...
729,frozen smoothie & ham,0.002800
738,almonds & olive oil,0.002800
140,escalope & vegetables mix,0.002800
771,tomato sauce & olive oil,0.002800


# FP Growth

In [29]:
from fpgrowth_py import fpgrowth
normalTransactions = [list(filter(lambda x: x != 'nan', it)) for it in transactions]

In [30]:
freqList, rules = fpgrowth(normalTransactions, minSupRatio=0.0028, minConf=0.2)
rules

[[{'mayonnaise'}, {'mineral water'}, 0.4782608695652174],
 [{'strong cheese'}, {'mineral water'}, 0.41379310344827586],
 [{'strong cheese'}, {'spaghetti'}, 0.4827586206896552],
 [{'bacon'}, {'eggs'}, 0.3384615384615385],
 [{'bacon'}, {'mineral water'}, 0.35384615384615387],
 [{'bacon'}, {'spaghetti'}, 0.36923076923076925],
 [{'green beans'}, {'spaghetti'}, 0.4],
 [{'green grapes'}, {'mineral water'}, 0.34328358208955223],
 [{'flax seed'}, {'green tea'}, 0.3382352941176471],
 [{'blueberries'}, {'spaghetti'}, 0.37681159420289856],
 [{'blueberries'}, {'mineral water'}, 0.391304347826087],
 [{'whole weat flour'}, {'mineral water'}, 0.36231884057971014],
 [{'zucchini'}, {'chocolate'}, 0.29577464788732394],
 [{'zucchini'}, {'spaghetti'}, 0.323943661971831],
 [{'zucchini'}, {'mineral water'}, 0.36619718309859156],
 [{'nonfat milk'}, {'spaghetti'}, 0.32051282051282054],
 [{'nonfat milk'}, {'mineral water'}, 0.48717948717948717],
 [{'cider'}, {'french fries'}, 0.27848101265822783],
 [{'cider'},

In [39]:
rules.sort(reverse=True, key=lambda x: x[2])

In [41]:
pd.DataFrame(rules)

,0,1,2
0,"{soup, frozen vegetables, milk}",{mineral water},0.766667
1,"{spaghetti, cooking oil, eggs}",{mineral water},0.709677
2,"{olive oil, frozen vegetables, chocolate}",{mineral water},0.700000
3,"{olive oil, frozen vegetables, milk}",{mineral water},0.694444
4,"{cereals, ground beef}",{spaghetti},0.676471
...,...,...,...
1555,{light mayo},{eggs},0.200980
1556,"{spaghetti, mineral water}",{frozen vegetables},0.200893
1557,"{spaghetti, chocolate}",{frozen vegetables},0.200680
1558,{soup},{chocolate},0.200528
